## Creando modelos basados en otros modelos

Hasta ahora hemos usado librerías como TensorFlow para crear nuestros propios modelos capa por capa desde cero.

Ahora vamos a realizar un proceso similar, excepto que la mayoría de las capas de nuestro modelo provendrán de [Kaggle](https://www.kaggle.com/models).

De hecho, vamos a utilizar dos modelos:
1. [ResNetV2](https://arxiv.org/abs/1603.05027) - una arquitectura de modelo de visión artificial basada en redes residuales (residual network).
2. [EfficientNet](https://arxiv.org/abs/1905.11946) - una arquitectura de visión artificial basada en redes convolucionales.

En algún momento, ambos modelos han logrado la tasa de error más baja en [ImageNet (ILSVRC-2012-CLS)](http://www.image-net.org/), el estándar de oro de los benchmarks de visión artificial.


pasos:
1. Ir a [https://www.kaggle.com/models](https://www.kaggle.com/models).
2. Elegir tu dominio de problema, por ejemplo, "Imagen" (estamos usando imágenes de alimentos).

> 🤔 **Pregunta:** *Veo muchas opciones para modelos de clasificación de imágenes, ¿cómo sé cuál es el mejor?*

Puedes ver una lista de modelos de última generación en [paperswithcode.com](https://www.paperswithcode.com), un recurso para recopilar los últimos resultados de trabajos de investigación en aprendizaje profundo que tienen implementaciones de código para los hallazgos que informan.

Dado que estamos trabajando con imágenes, nuestro objetivo son los [modelos que mejor se desempeñan en ImageNet](https://paperswithcode.com/sota/image-classification-on-imagenet).

   * La regla general aquí es, generalmente, los nombres con números más grandes significan modelos de mejor rendimiento. Por ejemplo, EfficientNetB4 tiene mejor capacidad de predicción que EfficientNetB0, sin embargo, los modelos más pequeños predicen más rápido.
     
7. Selecciona EfficientNet y en las variaciones de modelo, elige b0-feature-vector

> 🤔 **Pregunta:** *Pensé que estábamos haciendo clasificación de imágenes, ¿por qué elegimos vector de características y no clasificación?*

Gran observación. Aquí es donde entran en juego los diferentes tipos de transferencia de aprendizaje, como es, extracción de características y ajuste fino.

1. **Aprendizaje por transferencia "tal cual"** es cuando tomas un modelo preentrenado tal cual es y lo aplicas a tu tarea sin ningún cambio.

   * Por ejemplo, muchos modelos de visión artificial están preentrenados en el conjunto de datos en el conjunto de datos ImageNet, que contiene 1000 clases diferentes de imágenes. Esto significa que pasar una sola imagen a este modelo producirá 1000 valores de probabilidad de predicción diferentes (1 para cada clase).

    * Esto es útil si tienes 1000 clases de imágenes que te gustaría clasificar y todas son iguales a las clases de ImageNet, sin embargo, no es útil si quieres clasificar solo un pequeño subconjunto de clases (como 10 tipos diferentes de alimentos). Los modelos con `"/classification"` en su nombre proporcionan este tipo de funcionalidad.

2. **Aprendizaje por transferencia de extracción de características** es cuando tomas los patrones subyacentes (también llamados pesos) que un modelo preentrenado ha aprendido y ajustas sus salidas para que se adapten mejor a tu problema.

  * Por ejemplo, supongamos que el modelo preentrenado que estabas usando tenía 236 capas diferentes (EfficientNetB0 tiene 236 capas), pero la capa superior produce 1000 clases porque fue preentrenada en ImageNet. Para ajustar esto a tu propio problema, podrías eliminar la capa de activación original y reemplazarla con la tuya propia pero con el número correcto de clases de salida. La parte importante aquí es que **solo las primeras capas se vuelven entrenables, el resto permanece congelado**.

    * De esta manera, todos los patrones subyacentes permanecen en el resto de las capas y puedes utilizarlos para tu propio problema. Este tipo de aprendizaje por transferencia es muy útil cuando tus datos son similares a los datos con los que el modelo fue preentrenado.

3. **Aprendizaje por transferencia de ajuste fino (fine tuning)** es cuando tomas los patrones subyacentes (también llamados pesos) de un modelo preentrenado y los ajustas (afinas) a tu propio problema.

    * Esto generalmente significa entrenar **algunas, muchas o todas** las capas en el modelo preentrenado. Esto es útil cuando tienes un conjunto de datos grande (por ejemplo, más de 100 imágenes por clase) donde tus datos son ligeramente diferentes a los datos con los que el modelo original fue entrenado.

Un flujo de trabajo común es "congelar" todos los patrones aprendidos en las capas inferiores de un modelo preentrenado para que no se puedan entrenar. Y luego entrenar las 2-3 capas superiores para que el modelo preentrenado pueda ajustar sus salidas a tus datos personalizados (**extracción de características**).

Después de haber entrenado las 2-3 capas superiores, puedes gradualmente "descongelar" más y más capas y ejecutar el proceso de entrenamiento en tus propios datos para afinar aún más el modelo preentrenado.

> 🤔 **Pregunta:** *¿Por qué entrenar solo las 2-3 capas superiores en la extracción de características?*

Cuanto más baja es una capa en un modelo de visión artificial, es decir, cuanto más cerca está de la capa de entrada, mayores son las características que aprende. Por ejemplo, una capa inferior en un modelo de visión por computadora para identificar imágenes de gatos o perros podría aprender el contorno de las patas, mientras que, las capas más cercanas a la salida podrían aprender la forma de los dientes. A menudo, querrás que las características más grandes (los patrones aprendidos también se llaman características) permanezcan, ya que estas son similares para ambos animales, mientras que, las diferencias permanecen en las características más detalladas.

![image.png](transfer_learning.png)  
*Los diferentes tipos de aprendizaje por transferencia. Un modelo original, un modelo de extracción de características (solo cambia la cabecera) y un modelo de afinamiento fino (cambian muchas o todas las capas del modelo original).*

In [ ]:
import tensorflow as tf
print(tf.__version__)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
# USAREMOS TF_KERAS Para compatibilidad de resnet50v2
# pip install tf_keras
import tf_keras
print(tf_keras.__version__)

In [ ]:
# En primer lugar, vamos a cargar nuestro dataset

# Definir la forma de la imagen y el tamaño del lote
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

# Directorios para los conjuntos de datos de entrenamiento y prueba
train_dir = "./10_food_classes_10_percent/train/"
test_dir = "./10_food_classes_10_percent/test/"

# Crea el conjunto de datos de entrenamiento, utiliza los parámetros batch_size, image_size y label_mode='categorical'

# Crea el conjunto de datos de prueba de la  misma forma


In [ ]:
#importamos las librerías necesarias
import tensorflow as tf
import tensorflow_hub as hub
from tf_keras import layers

Ahora obtendremos las URLs de vector de características de dos arquitecturas, [EfficientNetB0 (2019)](https://www.kaggle.com/models/google/efficientnet-v2/tensorFlow2/imagenet1k-b0-feature-vector/2) y [ResNetV250 (2016)](https://www.kaggle.com/models/google/resnet-v2/tensorFlow2/50-feature-vector/1) desde Kaggle usando los pasos anteriores.

Obtenemos ambos porque vamos a compararlos para ver cuál se desempeña mejor en nuestros datos.

In [ ]:
resnet_url = "https://www.kaggle.com/models/google/resnet-v2/tensorFlow2/50-feature-vector/1"

efficientnet_url = "https://www.kaggle.com/models/google/efficientnet-v2/tensorFlow2/imagenet1k-b0-feature-vector/2"

In [ ]:
def create_model(model_url, num_classes=10):
  """Toma una URL de TensorFlow Hub y crea un modelo secuencial de Keras con ella.
  
  Args:
    model_url (str): Una URL de extracción de características de TensorFlow Hub.
    num_classes (int): Número de neuronas de salida en la capa de salida,
      debe ser igual al número de clases objetivo, por defecto 10.

  Returns:
    Un modelo secuencial de Keras sin compilar con model_url como capa
    de extracción de características y capa de salida densa con num_classes salidas.
  """
  model = ...
  return model

In [ ]:
# Crea un modelo basado en Resnet y compilalo


In [ ]:
# muestra el resumen del modelo

In [ ]:
# Entrena el modelo 5 épocas: Para el entrenamiento utiliza los parámetros
# steps_per_epoch=len(train_dataset),
# validation_data=test_dataset,
# validation_steps=len(test_dataset)


### ¡Impresionante!  

Parece que después de solo 5 épocas, el modelo de extracción de características **ResNetV250** logró superar con creces cualquiera de las arquitecturas que creamos, alcanzando aproximadamente un **90% de precisión en el conjunto de entrenamiento** y casi un **80% de precisión en el conjunto de prueba... ¡usando solo el 10% de las imágenes de entrenamiento!**

Esto demuestra el poder del **aprendizaje por transferencia**. Y es una de las principales razones por las que, siempre que intentes modelar tus propios conjuntos de datos, deberías investigar qué modelos preentrenados ya existen.

Vamos a revisar las curvas de entrenamiento de nuestro modelo utilizando nuestra función `plot_loss_curves`.


In [ ]:
import matplotlib.pyplot as plt

# Visualizar los datos de validación y entrenamiento por separado
def plot_loss_curves(history):
  """
  Devuelve curvas de pérdida separadas para las métricas de entrenamiento y validación.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Visualizar pérdida (loss)
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Pérdida (Loss)')
  plt.xlabel('Épocas')
  plt.legend()

  # Visualizar acierto (accuracy)
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Acierto (Accuracy)')
  plt.xlabel('Épocas')
  plt.legend();


In [ ]:
# imprime las curvas de entrenamiento

In [ ]:
# ¿Cuántos parámetros del modelo se han modificado?

Puedes ver el poder de **TensorFlow Hub** aquí. La capa de extracción de características tiene **23,564,800 parámetros**, que son patrones previamente aprendidos por el modelo en el conjunto de datos **ImageNet**. Como configuramos `trainable=False`, estos patrones permanecen congelados (no entrenables) durante el entrenamiento.

Esto significa que, durante el entrenamiento, el modelo actualiza los **20,490 parámetros** de la capa de salida para adaptarse a nuestro conjunto de datos.

Bien, hemos entrenado un modelo **ResNetV250**, ahora es momento de hacer lo mismo con el modelo **EfficientNetB0**.


In [ ]:
# Ahora crea un modelo basado en efficient_net, compila y entrena el modelo 5 épocas


In [ ]:
# muestra las curvas de entrenamiento

In [ ]:
# saca el resumen del modelo

¡Increíble! El modelo **EfficientNetB0** iguala al modelo **ResNetV250** con 1/4 de sus parámetros... nuevamente **usando solo el 10% de los datos de entrenamiento**!

Con solo unas pocas líneas de código, podemos aprovechar modelos de última generación y ajustarlos a nuestro propio caso de uso.

## Para bonus
### Puedes intentar utilizar transfer learning de ResNet50 o de EfficientNet con este dataset:
[razas de perros](https://www.kaggle.com/code/thiennguyen15/predicting-dog-species-using-resnet50)